In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import requests
from io import StringIO
m = Model('mip1')

Academic license - for non-commercial use only


### Data Wrangling

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
names = ['buying', 'maint','doors', 'persons', 'lug_boot', 'safety', 'class']
remote = requests.get(url).content
data_df = pd.read_csv(StringIO(remote.decode('utf-8')), names = names)


In [3]:
data_df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [4]:
def nullify(x):
    if x == '?':
        return np.nan
    else:
        return x

In [5]:
data_df = data_df.applymap(nullify)
data_df = data_df.dropna(axis = 0,how = 'any').reset_index(drop = True)

In [6]:
data_df = data_df.replace({'unacc': 0, 'acc': 1, 'good': 2, 'vgood': 3})
data_df['maint'] = data_df['maint'].replace({'vhigh': 0, 'high': 1, 'med': 2, 'low': 3})
data_df['buying'] = data_df['buying'].replace({'vhigh': 0, 'high': 1, 'med': 2, 'low': 3})
data_df['lug_boot'] = data_df['lug_boot'].replace({'small': 0, 'med': 1, 'big': 2})
data_df['safety'] = data_df['safety'].replace({'low': 0, 'med': 1, 'high': 2})
data_df['doors'] = data_df['doors'].replace({'2': 0, '3': 1, '4': 2, '5more': 3})
data_df['persons'] = data_df['persons'].replace({'2': 0, '4': 1, 'more': 2})
data_df = data_df.astype(int)
classification_X = data_df.drop(columns = ['class'])
classification_Y = data_df['class']

classification_X.head()

,buying,maint,doors,persons,lug_boot,safety
0,0,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,2
3,0,0,0,0,1,0
4,0,0,0,0,1,1


In [7]:
from sklearn.model_selection import train_test_split
X_train_df, X_test_df, Y_train_df, Y_test_df = train_test_split(classification_X, classification_Y, test_size = 0.20)

In [8]:
X_train_df = X_train_df.reset_index(drop = True)
Y_train_df = Y_train_df.reset_index(drop = True)
X_test_df = X_test_df.reset_index(drop = True)
Y_test_df = Y_test_df.reset_index(drop = True)

X_train = X_train_df.values
X_test = X_test_df.values
Y_train = Y_train_df.values
Y_test = Y_test_df.values

rows, cols = X_train.shape

### Determine the Depth

In [9]:
depth = 2

### CART Tree

In [10]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth = depth, max_leaf_nodes = 2**depth)
clf.fit(X_train_df, Y_train_df)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=4, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [11]:
total_nodes = clf.tree_.node_count
leaf_nodes = round(total_nodes / 2)
branch_nodes = total_nodes // 2

In [12]:
left_nodes = clf.tree_.children_left
right_nodes = clf.tree_.children_right
left_nodes = left_nodes[0:branch_nodes]
right_nodes = right_nodes[0:branch_nodes]

In [13]:
feature = clf.tree_.feature
initial_a = []
initial_a_tmp = []
for i in range(len(feature)):
    if feature[i] != -2 and feature[i+1] != -2:
        initial_a.append(feature[i])
    elif feature[i] != -2 and feature[i+1] == -2:
        initial_a_tmp.append(feature[i])
    else:
        continue
initial_a.extend(initial_a_tmp)
initial_a

[5, 3, 0]

In [14]:
threshold = clf.tree_.threshold
initial_b = []
initial_b_tmp = []
for i in range(len(threshold)):
    if threshold[i] != -2 and threshold[i+1] != -2:
        initial_b.append(threshold[i])
    elif threshold[i] != -2 and threshold[i+1] == -2:
        initial_b_tmp.append(threshold[i])
    else:
        continue
initial_b.extend(initial_b_tmp)

for idx, i in enumerate(initial_b):
    if i > 1:
        initial_b[idx] = 1

initial_b

[0.5, 0.5, 1]

In [15]:
clf.score(X_test_df, Y_test_df)

0.7861271676300579

### OCT

### Determine the alpha and K

In [16]:
alpha = 0.5
K = 4

In [17]:
max_diff = np.max(X_train, 0) - np.min(X_train, 0)
epsilon = np.ones([1, cols], dtype = int) * max_diff
for i in range(cols):
    old = 0
    for j in range(1, rows):
        diff = abs(X_train[j, i] - X_train[old ,i])
        old = j
        if diff < epsilon[0, i] and diff != 0:
            epsilon[0, i] = diff

epsilon = abs(epsilon)
epsilon

array([[1, 1, 1, 1, 1, 1]])

In [18]:
max_epsilon = np.max(epsilon)

n = rows
num_feature = cols

### Determine the Y matrix

In [19]:
Y = np.zeros([rows,K], dtype = int) - 1

Y[X_train_df.index, Y_train_df.astype(int)] = 1

Y

array([[ 1, -1, -1, -1],
       [-1,  1, -1, -1],
       [ 1, -1, -1, -1],
       ...,
       [ 1, -1, -1, -1],
       [ 1, -1, -1, -1],
       [ 1, -1, -1, -1]])

In [20]:
l = m.addVars(leaf_nodes, vtype = GRB.BINARY, name = "l")

z = m.addVars(n, leaf_nodes, vtype = GRB.BINARY, name = "z")

N_kt = m.addVars(K, leaf_nodes, vtype = GRB.INTEGER, name = "N_kt")

N_t = m.addVars(leaf_nodes, vtype = GRB.INTEGER, name = "N_t")

c_kt = m.addVars(K, leaf_nodes, vtype = GRB.BINARY, name = "c")

L = m.addVars(leaf_nodes, vtype = GRB.INTEGER, name = "L")

a = m.addVars(branch_nodes, num_feature, vtype = GRB.BINARY, name = 'a')

b = m.addVars(branch_nodes ,vtype = GRB.CONTINUOUS, name = "b")

d = m.addVars(branch_nodes ,vtype = GRB.BINARY, name = "d")

### Warm Start

In [21]:
# warm start using the results of CART algorithm

for i in range(branch_nodes):
    a[i, initial_a[i]].start = 1
    b[i].start = initial_b[i]

In [22]:
m.update()

In [23]:
m.setObjective(L.sum() + alpha * d.sum(), GRB.MINIMIZE)

### OCT Constraints

In [24]:
for i in range(branch_nodes):
    b[i].setAttr(GRB.Attr.LB, 0)
    m.addConstr(b[i] <= d[i])
    m.addConstr(a.sum(i, '*') == d[i])
    m.addConstr(d[i] == 1)
    m.addConstr(l[i] == 1)
    
for i in range(leaf_nodes):
    m.addConstr(L[i] >= 0)
    m.addConstr(N_t[i] == z.sum('*', i))
    m.addConstr(l[i] == c_kt.sum('*', i))
    m.addConstr(z.sum('*', i) >= l[i])
    for j in range(K):
        m.addConstr(L[i] >= N_t[i] - N_kt[j,i] - n * (1 - c_kt[j,i]))
        m.addConstr(L[i] <= N_t[i] - N_kt[j,i] + n * c_kt[j,i])
        m.addConstr(N_kt[j,i] == 1/2 * sum(z.select('*', i) * (Y[:,j] + 1)))

for i in range(n):
    m.addConstr(z.sum(i, '*') == 1)
    
for i in range (leaf_nodes):
    for j in range(n):
        m.addConstr(z[j,i] <= l[i])

all_branch_nodes = list(reversed(range(branch_nodes)))
depth_dict = {}
for i in range(depth):
    depth_dict[i] = sorted(all_branch_nodes[-2**i:])
    for j in range(2**i):
        all_branch_nodes.pop()

all_leaf_nodes = list(range(leaf_nodes))
branch_dict = {}
for i in range(branch_nodes):
    for k in range(depth):
        if i in depth_dict[k]:
            floor_len = len(depth_dict[k])
            step = 2**depth // floor_len
            sliced_leaf = [all_leaf_nodes[i:i+step] for i in range(0, 2**depth, step)]
            idx = depth_dict[k].index(i)
            branch_dict[i] = sliced_leaf[idx]
        else:
            continue
            
for j in range(n):
    for i in range(leaf_nodes):
        for k in range(branch_nodes):
            if i in branch_dict[k]:
                length = len(branch_dict[k])
                idx = branch_dict[k].index(i)
                if idx+1 <= length//2:
                    m.addConstr(sum(a.select(k, '*') * (X_train[j,:] + epsilon[0,:])) <= b[k] + (1 + max_epsilon) * (1 - z[j,i]))
                elif idx+1 > length//2:
                    m.addConstr(sum(a.select(k, '*') * X_train[j,:]) >= b[k] - (1 - z[j,i]))
            else:
                continue

In [25]:
m.optimize()

Optimize a model with 18042 rows, 5596 columns and 112097 nonzeros
Variable types: 3 continuous, 5593 integer (5569 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]

MIP start did not produce a new incumbent solution
MIP start violates constraint R6994 by 0.500000000

Presolve removed 9750 rows and 1895 columns
Presolve time: 1.15s
Presolved: 8292 rows, 3701 columns, 31911 nonzeros
Variable types: 0 continuous, 3701 integer (3683 binary)
Found heuristic solution: objective 313.5000000

Root relaxation: objective 1.500000e+00, 3432 iterations, 0.43 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.50000    0    5  313.50000    1.50000   100%     -    3s
Another try with MIP start
     0     0  209.00000    0 1224  313.50000  209.00000  33.3%   

In [26]:
for v in m.getVars():
    print(v.varName, v.x)

l[0] 1.0
l[1] 1.0
l[2] 1.0
l[3] 1.0
z[0,0] 1.0
z[0,1] 0.0
z[0,2] 0.0
z[0,3] 0.0
z[1,0] 0.0
z[1,1] 0.0
z[1,2] 0.0
z[1,3] 1.0
z[2,0] -0.0
z[2,1] -0.0
z[2,2] 1.0
z[2,3] -0.0
z[3,0] 0.0
z[3,1] 0.0
z[3,2] 0.0
z[3,3] 1.0
z[4,0] -0.0
z[4,1] -0.0
z[4,2] -0.0
z[4,3] 1.0
z[5,0] -0.0
z[5,1] -0.0
z[5,2] 1.0
z[5,3] -0.0
z[6,0] 0.0
z[6,1] 1.0
z[6,2] -0.0
z[6,3] -0.0
z[7,0] -0.0
z[7,1] -0.0
z[7,2] 1.0
z[7,3] -0.0
z[8,0] 0.0
z[8,1] 0.0
z[8,2] 0.0
z[8,3] 1.0
z[9,0] 0.0
z[9,1] 0.0
z[9,2] 0.0
z[9,3] 1.0
z[10,0] -0.0
z[10,1] -0.0
z[10,2] -0.0
z[10,3] 1.0
z[11,0] 0.0
z[11,1] 0.0
z[11,2] 0.0
z[11,3] 1.0
z[12,0] 0.0
z[12,1] 1.0
z[12,2] -0.0
z[12,3] -0.0
z[13,0] 0.0
z[13,1] 0.0
z[13,2] 0.0
z[13,3] 1.0
z[14,0] -0.0
z[14,1] -0.0
z[14,2] -0.0
z[14,3] 1.0
z[15,0] -0.0
z[15,1] -0.0
z[15,2] 1.0
z[15,3] -0.0
z[16,0] 0.0
z[16,1] 1.0
z[16,2] -0.0
z[16,3] -0.0
z[17,0] -0.0
z[17,1] -0.0
z[17,2] 1.0
z[17,3] 0.0
z[18,0] -0.0
z[18,1] -0.0
z[18,2] 1.0
z[18,3] -0.0
z[19,0] -0.0
z[19,1] -0.0
z[19,2] 1.0
z[19,3] -0.0
z[20,0] 1

z[386,3] 1.0
z[387,0] 1.0
z[387,1] 0.0
z[387,2] -0.0
z[387,3] -0.0
z[388,0] -0.0
z[388,1] -0.0
z[388,2] -0.0
z[388,3] 1.0
z[389,0] 0.0
z[389,1] 1.0
z[389,2] -0.0
z[389,3] -0.0
z[390,0] -0.0
z[390,1] -0.0
z[390,2] 1.0
z[390,3] -0.0
z[391,0] -0.0
z[391,1] -0.0
z[391,2] -0.0
z[391,3] 1.0
z[392,0] 0.0
z[392,1] 1.0
z[392,2] -0.0
z[392,3] -0.0
z[393,0] 0.0
z[393,1] 0.0
z[393,2] 0.0
z[393,3] 1.0
z[394,0] -0.0
z[394,1] -0.0
z[394,2] 1.0
z[394,3] -0.0
z[395,0] 0.0
z[395,1] 1.0
z[395,2] -0.0
z[395,3] -0.0
z[396,0] -0.0
z[396,1] -0.0
z[396,2] 1.0
z[396,3] -0.0
z[397,0] 0.0
z[397,1] 1.0
z[397,2] -0.0
z[397,3] -0.0
z[398,0] 0.0
z[398,1] 1.0
z[398,2] -0.0
z[398,3] -0.0
z[399,0] 0.0
z[399,1] 1.0
z[399,2] -0.0
z[399,3] -0.0
z[400,0] 0.0
z[400,1] 1.0
z[400,2] -0.0
z[400,3] -0.0
z[401,0] 0.0
z[401,1] 0.0
z[401,2] 0.0
z[401,3] 1.0
z[402,0] -0.0
z[402,1] -0.0
z[402,2] -0.0
z[402,3] 1.0
z[403,0] 0.0
z[403,1] 0.0
z[403,2] 0.0
z[403,3] 1.0
z[404,0] -0.0
z[404,1] -0.0
z[404,2] 1.0
z[404,3] -0.0
z[405,0] 0.0
z

z[736,3] -0.0
z[737,0] -0.0
z[737,1] -0.0
z[737,2] -0.0
z[737,3] 1.0
z[738,0] 0.0
z[738,1] 1.0
z[738,2] -0.0
z[738,3] -0.0
z[739,0] -0.0
z[739,1] -0.0
z[739,2] -0.0
z[739,3] 1.0
z[740,0] 1.0
z[740,1] 0.0
z[740,2] -0.0
z[740,3] -0.0
z[741,0] -0.0
z[741,1] -0.0
z[741,2] 1.0
z[741,3] -0.0
z[742,0] 0.0
z[742,1] 1.0
z[742,2] -0.0
z[742,3] -0.0
z[743,0] 0.0
z[743,1] 0.0
z[743,2] 0.0
z[743,3] 1.0
z[744,0] 0.0
z[744,1] 0.0
z[744,2] 0.0
z[744,3] 1.0
z[745,0] 0.0
z[745,1] 0.0
z[745,2] 0.0
z[745,3] 1.0
z[746,0] -0.0
z[746,1] -0.0
z[746,2] 1.0
z[746,3] -0.0
z[747,0] -0.0
z[747,1] -0.0
z[747,2] 1.0
z[747,3] -0.0
z[748,0] -0.0
z[748,1] -0.0
z[748,2] 1.0
z[748,3] -0.0
z[749,0] 0.0
z[749,1] 0.0
z[749,2] 0.0
z[749,3] 1.0
z[750,0] 0.0
z[750,1] 0.0
z[750,2] 0.0
z[750,3] 1.0
z[751,0] 1.0
z[751,1] 0.0
z[751,2] 0.0
z[751,3] 0.0
z[752,0] -0.0
z[752,1] -0.0
z[752,2] -0.0
z[752,3] 1.0
z[753,0] 1.0
z[753,1] 0.0
z[753,2] -0.0
z[753,3] -0.0
z[754,0] 1.0
z[754,1] 0.0
z[754,2] -0.0
z[754,3] -0.0
z[755,0] -0.0
z[755

z[1116,0] 0.0
z[1116,1] 0.0
z[1116,2] 0.0
z[1116,3] 1.0
z[1117,0] -0.0
z[1117,1] -0.0
z[1117,2] -0.0
z[1117,3] 1.0
z[1118,0] -0.0
z[1118,1] -0.0
z[1118,2] 1.0
z[1118,3] -0.0
z[1119,0] 0.0
z[1119,1] 0.0
z[1119,2] 0.0
z[1119,3] 1.0
z[1120,0] -0.0
z[1120,1] -0.0
z[1120,2] 1.0
z[1120,3] -0.0
z[1121,0] 0.0
z[1121,1] 0.0
z[1121,2] 0.0
z[1121,3] 1.0
z[1122,0] -0.0
z[1122,1] -0.0
z[1122,2] -0.0
z[1122,3] 1.0
z[1123,0] -0.0
z[1123,1] -0.0
z[1123,2] 1.0
z[1123,3] -0.0
z[1124,0] 0.0
z[1124,1] 0.0
z[1124,2] 0.0
z[1124,3] 1.0
z[1125,0] -0.0
z[1125,1] -0.0
z[1125,2] -0.0
z[1125,3] 1.0
z[1126,0] -0.0
z[1126,1] -0.0
z[1126,2] -0.0
z[1126,3] 1.0
z[1127,0] -0.0
z[1127,1] -0.0
z[1127,2] 1.0
z[1127,3] -0.0
z[1128,0] 0.0
z[1128,1] 0.0
z[1128,2] 0.0
z[1128,3] 1.0
z[1129,0] 0.0
z[1129,1] 0.0
z[1129,2] 0.0
z[1129,3] 1.0
z[1130,0] 0.0
z[1130,1] 0.0
z[1130,2] 0.0
z[1130,3] 1.0
z[1131,0] 0.0
z[1131,1] 0.0
z[1131,2] 0.0
z[1131,3] 1.0
z[1132,0] 0.0
z[1132,1] 1.0
z[1132,2] -0.0
z[1132,3] -0.0
z[1133,0] 0.0
z[1133,1

In [27]:
print('Obj:', m.objVal)

Obj: 313.5


### Obtain the Tree Structure

In [28]:
# Obtain the coefficients of a and b
coff_a = np.zeros([branch_nodes, num_feature], dtype = int)
# coff_b = np.zeros(branch_nodes, dtype = int)
coff_b = np.zeros(branch_nodes)

for i in range(branch_nodes):
    tmp1 = m.getVarByName('b' + '[' + str(i) + ']')
#     coff_b[i] = int(tmp1.x)
    coff_b[i] = tmp1.x
    for j in range(num_feature):
        tmp2 = m.getVarByName('a' + '[' + str(i) + ',' + str (j) + ']')
        coff_a[i,j] = int(tmp2.x)



In [29]:
# Obtain the labels of leaf nodes
labels = np.zeros(leaf_nodes, dtype = int) - 1
coff_c = np.zeros([K, leaf_nodes], dtype = int)

for i in range(K):
    for j in range(leaf_nodes):
        tmp3 = m.getVarByName('c' + '[' + str(i) + ',' + str (j) + ']')
        coff_c[i,j] = int(tmp3.x)

k_idx, t_idx = np.where(coff_c == 1)
# for i in range(leaf_nodes):
for i in range(len(k_idx)):
    labels[t_idx[i]] = k_idx[i]

### Test Data

In [30]:
t_rows, t_cols = X_test.shape

In [31]:
tmp = np.zeros([t_rows, 1], dtype = int)
Y_predict = np.hstack((np.reshape(Y_test_df.values, (t_rows, 1)), tmp))

In [32]:
num_nodes = 0
for i in range(branch_nodes): 
    tmp4 = m.getVarByName('d' + '[' + str(i) + ']')
    num_nodes += int(tmp4.x)

In [33]:
init = np.array([], dtype = int).reshape(0, num_feature)
nodes = {}
for i in range(num_nodes * 2):
    nodes[i] = init

# split
for i in range(t_rows):
    if np.dot(coff_a[0,:], np.transpose(X_test[i,:])) < coff_b[0]:
        nodes[0] = np.vstack([X_test[i,:], nodes[0]])
        if np.dot(coff_a[1,:], np.transpose(X_test[i,:])) < coff_b[1]:
            nodes[2] = np.vstack([X_test[i,:], nodes[2]])
            Y_predict[i,1] = labels[0]
            
        elif np.dot(coff_a[1,:], np.transpose(X_test[i,:])) >= coff_b[1]:
            nodes[3] = np.vstack([X_test[i,:], nodes[3]])
            Y_predict[i,1] = labels[1]
            
    elif np.dot(coff_a[0,:], np.transpose(X_test[i,:])) >= coff_b[0]:
        nodes[1] = np.vstack([X_test[i,:], nodes[1]])
        if np.dot(coff_a[2,:], np.transpose(X_test[i,:])) < coff_b[2]:
            nodes[4] = np.vstack([X_test[i,:], nodes[4]])
            Y_predict[i,1] = labels[2]
            
        elif np.dot(coff_a[2,:], np.transpose(X_test[i,:])) >= coff_b[2]:
            nodes[5] = np.vstack([X_test[i,:], nodes[5]])
            Y_predict[i,1] = labels[3]



### Test Accuracy

In [34]:
1 - sum(np.minimum(abs(Y_predict[:,1] - Y_predict[:,0]), 1)) / t_rows

0.791907514450867